In [1]:
import urllib
import json
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import string
import nltk
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
import pickle

In [2]:
#Task1
def getpage(num):
    url = "https://community.smartthings.com/latest?no_definitions=true&page="+str(num)
    r = requests.get(url)
    soup = BeautifulSoup(r.text, "lxml")
    soup_all = soup.findAll('span', class_='link-top-line')
    title = []
    url = []
    for i in soup_all:
        post = i.find(class_= "title raw-link raw-topic-link")
        title.append(post.text)
        url.append(post.get('href'))
        #print(url)
    tag = []
    tags_all = soup.findAll(class_="link-bottom-line")
    for i in tags_all:
        tag.append(i.find('span',class_='category-name').text)
    subtag = []
    subtags_all = soup.findAll('div',class_='discourse-tags')
    for i in subtags_all:
        discourse_tag = i.find('a',class_ = "discourse-tag")
        if discourse_tag:
            subtag.append(discourse_tag.text)
        else:
            subtag.append(None)
    data = pd.DataFrame({'title':title,'tag':tag,'subtag':subtag,'url':url})
    return data

In [3]:
#Task2
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()
def remove_html(text):
    sup = BeautifulSoup(text,'lxml')
    html_free = sup.get_text()
    return html_free

def remove_punc(text):
    no_punc = "".join([c for c in text if c not in string.punctuation])
    return no_punc
def remove_stopwords(text):
    words = [w for w in text if w not in stopwords.words('english')]
    return words
def word_lemmatizer(text):
    lem_text = [lemmatizer.lemmatize(i) for i in text]
    return lem_text
def word_stemmer(text):
    stem_text = [stemmer.stem(i) for i in text]
    return stem_text

def read_from_url(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    containers = soup.findAll("div", {"class":"topic-body crawler-post"})
    df = pd.DataFrame(columns=['user', 'content','raw_content'])
    count = 0
    for container in containers:

        user_container = container.findAll("span", {"itemprop":"name"})
        user = user_container[0].text
        #print("User: " + user.lower())

        content_container = container.findAll("div", {"class":"post"})
        """
        This if statement should be removed once infinite scorlling bar is handled
        """
        if content_container:
            content = remove_html(content_container[0].text)
        #print("Content: ")
            #print(content)

            dfcontent = (content.lower()).replace("\t","").replace("\n"," ").replace("https ", "https")\
                .replace("…","").replace("we’re", "we are").replace("“","").replace("”","").replace("i’ll", "i will")
            #print(dfcontent)
        #print("\n")
        
        #else:
            #print("content empty, perhaps because infinite scrolling bar")

        gruber = re.compile(r"""(?i)\b((?:https?:(?:/{1,3}|[a-z0-9%])|[a-z0-9.\-]+[.](?:com|net|org|edu|
        gov|mil|aero|asia|biz|cat|coop|info|int|jobs|mobi|museum|name|post|pro|tel|travel|xxx|ac|ad|ae|af|
        ag|ai|al|am|an|ao|aq|ar|as|at|au|aw|ax|az|ba|bb|bd|be|bf|bg|bh|bi|bj|bm|bn|bo|br|bs|bt|bv|bw|by|bz
        |ca|cc|cd|cf|cg|ch|ci|ck|cl|cm|cn|co|cr|cs|cu|cv|cx|cy|cz|dd|de|dj|dk|dm|do|dz|ec|ee|eg|eh|er|es|et
        |eu|fi|fj|fk|fm|fo|fr|ga|gb|gd|ge|gf|gg|gh|gi|gl|gm|gn|gp|gq|gr|gs|gt|gu|gw|gy|hk|hm|hn|hr|ht|hu|id
        |ie|il|im|in|io|iq|ir|is|it|je|jm|jo|jp|ke|kg|kh|ki|km|kn|kp|kr|kw|ky|kz|la|lb|lc|li|lk|lr|ls|lt|lu
        |lv|ly|ma|mc|md|me|mg|mh|mk|ml|mm|mn|mo|mp|mq|mr|ms|mt|mu|mv|mw|mx|my|mz|na|nc|ne|nf|ng|ni|nl|no|np
        |nr|nu|nz|om|pa|pe|pf|pg|ph|pk|pl|pm|pn|pr|ps|pt|pw|py|qa|re|ro|rs|ru|rw|sa|sb|sc|sd|se|sg|sh|si|sj|
        Ja|sk|sl|sm|sn|so|sr|ss|st|su|sv|sx|sy|sz|tc|td|tf|tg|th|tj|tk|tl|tm|tn|to|tp|tr|tt|tv|tw|tz|ua|ug|
        uk|us|uy|uz|va|vc|ve|vg|vi|vn|vu|wf|ws|ye|yt|yu|za|zm|zw)/)(?:[^\s()<>{}\[\]]+|\([^\s()]*?\([^\s()]+
        \)[^\s()]*?\)|\([^\s]+?\))+(?:\([^\s()]*?\([^\s()]+\)[^\s()]*?\)|\([^\s]+?\)|[^\s`!()\[\]{};:'".,<>
        ?«»“”‘’])|(?:(?<!@)[a-z0-9]+(?:[.\-][a-z0-9]+)*[.](?:com|net|org|edu|gov|mil|aero|asia|biz|cat|coop
        |info|int|jobs|mobi|museum|name|post|pro|tel|travel|xxx|ac|ad|ae|af|ag|ai|al|am|an|ao|aq|ar|as|at|au
        |aw|ax|az|ba|bb|bd|be|bf|bg|bh|bi|bj|bm|bn|bo|br|bs|bt|bv|bw|by|bz|ca|cc|cd|cf|cg|ch|ci|ck|cl|cm|cn|
        co|cr|cs|cu|cv|cx|cy|cz|dd|de|dj|dk|dm|do|dz|ec|ee|eg|eh|er|es|et|eu|fi|fj|fk|fm|fo|fr|ga|gb|gd|ge|g
        f|gg|gh|gi|gl|gm|gn|gp|gq|gr|gs|gt|gu|gw|gy|hk|hm|hn|hr|ht|hu|id|ie|il|im|in|io|iq|ir|is|it|je|jm|jo
        |jp|ke|kg|kh|ki|km|kn|kp|kr|kw|ky|kz|la|lb|lc|li|lk|lr|ls|lt|lu|lv|ly|ma|mc|md|me|mg|mh|mk|ml|mm|mn|
        mo|mp|mq|mr|ms|mt|mu|mv|mw|mx|my|mz|na|nc|ne|nf|ng|ni|nl|no|np|nr|nu|nz|om|pa|pe|pf|pg|ph|pk|pl|pm|p
        n|pr|ps|pt|pw|py|qa|re|ro|rs|ru|rw|sa|sb|sc|sd|se|sg|sh|si|sj|Ja|sk|sl|sm|sn|so|sr|ss|st|su|sv|sx|sy
        |sz|tc|td|tf|tg|th|tj|tk|tl|tm|tn|to|tp|tr|tt|tv|tw|tz|ua|ug|uk|us|uy|uz|va|vc|ve|vg|vi|vn|vu|wf|ws|
        ye|yt|yu|za|zm|zw)\b/?(?!@)))""")
        split_dfcontent = gruber.split(dfcontent)

        raw_dfcontent = list(filter(None, dfcontent.split("\n")))

        for i in range(0, len(split_dfcontent), 2):
            split_dfcontent[i] = remove_punc(split_dfcontent[i])

        final_dfcontent = "".join(split_dfcontent)

        df.loc[count] = [user.lower()] + [(' '.join(final_dfcontent.split())).lower()]+[raw_dfcontent]
        count += 1

        #print((' '.join(final_dfcontent.split())).lower())
        #print("\n")
    df['stem'] = df['content']
    #print(df)
    for i in range(len(containers)):
        #print(df['Content'][i])
        df['stem'][i] = re.split(r'\s{1,}', df['content'][i])
        #df['origin_content'][i] = list(filter(None, content))
    df['stem'] = df['stem'].apply(lambda x : remove_stopwords(x))
    """
    
    df['stem']=df['stem'].apply(lambda x: word_lemmatizer(x))
    """
    
    df['stem'] = df['stem'].apply(lambda x: word_stemmer(x))
    return df

In [4]:
c = read_from_url("https://community.smartthings.com/t/fire-hd-8-tablets-on-sale-amazon-best-buy-june-29-2020/198367")

In [5]:
dflist = []
for i in range(1000):
    df = getpage(i)
    dflist.append(df)

In [6]:
data = pd.concat(dflist).reset_index().drop(columns = ['index'])

In [7]:
data['starter_content'] = ''
data['starter_stem'] = ''
data['starter_raw_content'] = ''
data['reply_content'] = ''
data['reply_stem'] = ''
data['reply_raw_content'] = ''
data['category'] = ''

In [ ]:
for i in range(len(data)):
    subdata = read_from_url(data['url'][i])
    starter_content = ''
    starter_stem = []
    starter_raw_content = []
    reply_raw_content = []
    reply_content=''
    reply_stem = []
    for k in range(len(subdata)):
        if subdata['user'][k] == subdata['user'][0]:
            starter_content += subdata['content'][k]
            starter_raw_content += subdata['raw_content'][k]
            starter_stem += subdata['stem'][k]
        else:
            reply_content += subdata['content'][k]
            reply_raw_content += subdata['raw_content'][k]
            reply_stem += subdata['stem'][k]    
    data['starter_content'][i] = starter_content
    data['starter_raw_content'][i] = starter_raw_content
    data['starter_stem'][i] = starter_stem
    data['reply_content'][i] = reply_content
    data['reply_raw_content'][i] = reply_raw_content
    data['reply_stem'][i] = reply_stem

In [ ]:
data['tag'].unique()

In [ ]:
data.to_csv('data.csv')

In [ ]:
data.head()